<a href="https://colab.research.google.com/github/RexGod/DataMining_Pgu_Nueral_Network/blob/main/finalDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

In [10]:
pathImages = '/content/drive/MyDrive/tits_pic'

In [11]:
pathManual = '/content/drive/MyDrive/Radiology_manual_annotations.xlsx'
manualRadio = pd.read_excel(pathManual)

In [12]:
manualRadio= manualRadio.rename(columns={'Pathology Classification/ Follow up':'classification'})

In [13]:
manualRadio = manualRadio[manualRadio['Type']=='CESM']

In [14]:
import os
images = os.listdir(pathImages)
loaded_images = []
#filename_txt = []
for filename in images:
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_list = plt.imread(os.path.join(pathImages, filename))
        loaded_images.append(image_list)
        #filename_txt.append(filename)

In [15]:
manualRadio = manualRadio.sort_values(by=['Image_name'])

In [16]:
validation_data_size = int(len(loaded_images)*0.12)
test_data_size = int(len(loaded_images)*0.13)

In [17]:
random_select = np.random.choice(len(loaded_images), size=validation_data_size, replace=False)

In [18]:
validation_data_x = np.array(loaded_images)[random_select]
validation_data_y = manualRadio.iloc[random_select]
loaded_images = np.delete(loaded_images, random_select, axis=0)
manualRadio=manualRadio.drop(manualRadio.index[random_select])

In [19]:
random_select = np.random.choice(len(loaded_images), size=test_data_size, replace=False)

In [20]:
test_data_x = np.array(loaded_images)[random_select]
test_data_y = manualRadio.iloc[random_select]
loaded_images = np.delete(loaded_images, random_select, axis=0)
manualRadio=manualRadio.drop(manualRadio.index[random_select])

In [21]:
train_data_x = np.array(loaded_images)
train_data_y = manualRadio[["Image_name" , "classification"]]

In [22]:
import tensorflow as tf
import keras

In [23]:
label_mapping = {'Benign': 0, 'Normal': 1, 'Malignant': 2}

In [24]:
integer_labels = train_data_y['classification'].map(label_mapping)
integer_labels_valid = validation_data_y['classification'].map(label_mapping)
y_train_to_categorical = keras.utils.to_categorical(integer_labels, num_classes=len(label_mapping), dtype='float32')
y_validation_to_categorical = keras.utils.to_categorical(integer_labels_valid, num_classes=len(label_mapping), dtype='float32')

In [25]:
from tensorflow.keras.applications.resnet import ResNet101,ResNet152 , preprocess_input , decode_predictions

In [26]:
train_images_rgb = np.repeat(train_data_x[..., np.newaxis], 3, -1)[..., 0]
test_images_rgb = np.repeat(test_data_x[..., np.newaxis], 3, -1)
validation_images_rgb = np.repeat(validation_data_x[..., np.newaxis], 3, -1)[..., 0]


In [27]:
train_images_rgb = train_images_rgb/255
preprocess_images_test = preprocess_input(test_images_rgb)
validation_images_rgb = validation_images_rgb/255

In [28]:
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

234698864/234698864 [==============================] - 13s 0us/step


In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D , Conv2D , Flatten , Dropout , MaxPooling2D , BatchNormalization
from keras import layers
from tensorflow.keras.optimizers import Adam,Adamax


In [48]:
model = Sequential()

In [49]:
model.add(base_model)


In [50]:
for layer in base_model.layers[:150]:
    layer.trainable = False
for layer in base_model.layers[150:]:
    layer.trainable = True

In [51]:
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256 , activation='relu'))
model.add(Dense(128 , activation='relu'))
model.add(layers.Dropout(0.2))
model.add(Dense(64 , activation='relu'))
model.add(layers.Dropout(0.3))
model.add(Dense(3,activation = 'softmax'))

In [52]:
model.compile(optimizer=Adam(learning_rate=0.0002), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
from tensorflow.keras.callbacks import EarlyStopping

In [54]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

In [ ]:
nemudar = model.fit(train_images_rgb, y_train_to_categorical,
                    validation_data=(validation_images_rgb, y_validation_to_categorical),
                    epochs=30,
                    callbacks = [early_stopping]
                    )


Epoch 1/30
